### Check type of GPU and VRAM available.

In [1]:
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

Tesla T4, 15109 MiB, 15109 MiB


### Install Dependencies

In [2]:
# Clone repos we need
!git clone https://github.com/Jan-Oliver/profaile-pic-dev.git
!git clone https://github.com/huggingface/diffusers.git

# Handle all of the dependencies
!cd /content/profaile-pic-dev && chmod +x dreambooth/setup_dreambooth_env.sh
!cd /content/profaile-pic-dev && dreambooth/setup_dreambooth_env.sh
!cd /content/profaile-pic-dev && chmod +x inference/setup_inference_env.sh
!cd /content/profaile-pic-dev && inference/setup_inference_env.sh

Cloning into 'profaile-pic-dev'...
remote: Enumerating objects: 61, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 61 (delta 28), reused 53 (delta 20), pack-reused 0
Unpacking objects: 100% (61/61), done.
Cloning into 'diffusers'...
remote: Enumerating objects: 15199, done.
remote: Counting objects: 100% (163/163), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 15199 (delta 110), reused 114 (delta 78), pack-reused 15036
Receiving objects: 100% (15199/15199), 10.28 MiB | 5.17 MiB/s, done.
Resolving deltas: 100% (10482/10482), done.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 182 kB 21.4 MB/s 
     |████████████████████████████████| 18.7 MB 48 kB/s 
     |████████████████████████████████| 143 kB 34.2 MB/s 
     |████████████████████████████████| 5.8 MB 66.9 MB/s 
     |██████████

### Set up W&B and GDrive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import wandb
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

### Create Run and load model from W&B

In [7]:
# Name for W&B
PROJECT_NAME = "stable-diffusion2"
# Increment
GROUP_NAME = "exp-finetuning-Nimages-all"
run = wandb.init(project=PROJECT_NAME, group=GROUP_NAME, job_type="prompt_engineering")

In [13]:
import torch, os
from torch import autocast
from diffusers import StableDiffusionPipeline, DDIMScheduler, EulerDiscreteScheduler
from IPython.display import display

In [14]:
ARTIFACT_DIR = "/content/finetuned_model_checkpoint"
MODEL_NAME = "model.ckpt"
MODEL_PATH = os.path.join(ARTIFACT_DIR, MODEL_NAME)

run = wandb.init()
artifact = run.use_artifact('janoliver/stable-diffusion2/SD2.1_finetuned_jseidenfuss:v0', type='model')
artifact_dir = artifact.download(ARTIFACT_DIR)
run.finish()

# As I forgot to also save the checkpoint configuration yaml file we have to download the correct version manually
!wget https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference.yaml
!mv /content/v2-inference.yaml /content/finetuned_model_checkpoint/

wandb: Downloading large artifact SD2.1_finetuned_jseidenfuss:v0, 4921.17MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:4:27.5


--2022-12-12 08:14:20--  https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1789 (1.7K) [text/plain]
Saving to: ‘v2-inference.yaml’

v2-inference.yaml   100%[===================>]   1.75K  --.-KB/s    in 0s      

2022-12-12 08:14:20 (28.0 MB/s) - ‘v2-inference.yaml’ saved [1789/1789]



In [16]:
DUMP_PATH = "/content/finetuned_model_diffuser"
CONFIG_FILE="/content/finetuned_model_checkpoint/v2-inference.yaml"
!python /content/profaile-pic-dev/utils/convert_original_stable_diffusion_to_diffusers.py \
  --checkpoint_path=$MODEL_PATH \
  --original_config_file=$CONFIG_FILE \
  --dump_path=$DUMP_PATH \
  --prediction_type="v-prediction" \
  --image_size=512

Downloading: 100% 633/633 [00:00<00:00, 610kB/s]
Downloading: 100% 1.36G/1.36G [00:15<00:00, 86.2MB/s]
^C


### Run inference

In [ ]:
from PIL import Image

PRECISION = "no" # We don't use any type of optimization for this run

In [ ]:
def image_grid(imgs, rows, cols):
  assert len(imgs) == rows*cols

  w, h = imgs[0].size
  grid = Image.new('RGB', size=(cols*w, rows*h))
  grid_w, grid_h = grid.size
  
  for i, img in enumerate(imgs):
      grid.paste(img, box=(i%cols*w, i//cols*h))
  return grid

def init_table():
  # log inference results as a Table to the run workspace
  columns=["prompt", "n_prompt", "guidance_scale", "num_inference_steps", "height", "width"]
  table_instance = wandb.Table(columns = columns)
  return table_instance

def run_and_log_pipeline(prompt, nprompt, guid_scale, ninf_steps, h, w):
  # Make individual calls determninistic
  SEED = 52362
  g_cuda = torch.Generator(device='cuda').manual_seed(SEED)

  images = pipe(
    prompt,
    height=h,
    width=w,
    negative_prompt=nprompt,
    num_images_per_prompt=8,
    num_inference_steps=ninf_steps,
    guidance_scale=guid_scale,
    generator=g_cuda
  ).images

  # Create table
  columns=["seed", "prompt", "n_prompt", "guidance_scale", "num_inference_steps", "height", "width", "Images"]
  table_instance = wandb.Table(columns = columns)
  wandb_images = []
  for image in images:
    wandb_images.append(wandb.Image(image))
  row = [prompt, nprompt, guid_scale, ninf_steps, h, w, wandb_images]
  table_instance.add_date(*row)
  run.log({"prompt_engineering": table_instance})  
  return images

In [ ]:
import torch
from torch import autocast
from diffusers import StableDiffusionPipeline, DDIMScheduler, EulerDiscreteScheduler
from IPython.display import display

TABLE_INSTANCE_WANDB = init_table()

scheduler_euler = EulerDiscreteScheduler.from_pretrained(MODEL_NAME, subfolder="scheduler")
scheduler_ddim = DDIMScheduler.from_pretrained(MODEL_NAME,subfolder="scheduler")

# First for Euler
if PRECISION == "fp16":
  pipe = StableDiffusionPipeline.from_pretrained(MODEL_PATH, scheduler=scheduler_euler, safety_checker=None, torch_dtype=torch.float16).to("cuda")
else:
  pipe = StableDiffusionPipeline.from_pretrained(MODEL_PATH, scheduler=scheduler_euler, safety_checker=None, torch_dtype=torch.float32).to("cuda")

# Then for DDIM
g_cuda = torch.Generator(device='cuda').manual_seed(52362)

prompt = "photo of face and shoulders of jseidenfuss man in a blue suit from the front, front view, closeup, centered frame, symmetric, studio lighting, clear and realistic face, uhd faces, pexels, 85mm, casual pose, 35mm film roll photo, hard light, detailed skin texture, masterpiece, sharp focus, pretty, lovely, adorable, attractive, hasselblad, candid street portrait"
negative_prompt = "blender, ugly, multiple hands, bad anatomy, bad proportions, unrealistic, full body, cropped, lowres, poorly drawn face, out of frame, poorly drawn hands, double, blurred, disfigured, deformed, repetitive, black and white "
guidance_scale = 7
num_inference_steps = 70
height = 512
width = 512


images = run_and_log_pipeline(prompt, 
                              negative_prompt, 
                              guidance_scale, 
                              num_inference_steps, 
                              height, 
                              width)
for image in images:
  display(image)
del pipe

### Legacy

In [ ]:
# Run over multiple parameters and generate a grid.
"""grids = []
for num_inference_steps in [50, 70, 90]:
  with autocast("cuda"), torch.inference_mode():
      images = pipe(
          prompt,
          height=height,
          width=width,
          negative_prompt=negative_prompt,
          num_images_per_prompt=num_samples,
          num_inference_steps=num_inference_steps,
          guidance_scale=guidance_scale,
          generator=g_cuda
      ).images
      grid = image_grid(images, 1, num_samples)
      grids.append(grid)
"""